In [14]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np
from time import sleep
import re
from fuzzywuzzy import fuzz
import joblib
import pickle

In [15]:
import sys
#sys.path.insert(1, '/content/drive/MyDrive/Credentials/')
from credit_x import *

import spotipy
import json
from spotipy.oauth2 import SpotifyClientCredentials


#Initialize SpotiPy with user credentias
sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(client_id= client_id,
                                                           client_secret= client_secret))

playlist_id="2cqiYaM8wr1fFYFZbCXEwh"
username="11131870089?si=dSbwy5RXRVeQwdBpDHfOWQ"
username="11131870089"

# LETS IMPROVE YOUR FAVORITE PLAYLIST ON SPOTIFY

In [18]:
def user_interface():
    username=input("Please enter your username: ")
    print()
    playlist_id=input("Please enter your most favorite playlist you want to improve: ")
    
    #(1) Get audiofeatures of your playlist
    track_feature=get_audio_features(username, playlist_id)
    
    #(2) Get audiofeatures_df of your playlist
    song_df=get_audio_features_df(track_feature)
    
    #(3) Get cluster for your playlist
    song_df_cluster=cluster_playlist(song_df)
    
    # TEST (4) Get song recommendations
    recommendations(song_df_cluster)

In [19]:
user_interface()

Please enter your username: 11131870089

Please enter your most favorite playlist you want to improve: 2cqiYaM8wr1fFYFZbCXEwh

Here are my recommendations for you improving your playlist: 


I Didn't Know  by  Jean Goldkette and his Orchestra
https://open.spotify.com/track/7AO6HiSVKT8WjYGc5Oh9ws
____________________________________________

If I Were King of the Forest - Alternate Vocal Tag  by  Bert Lahr
https://open.spotify.com/track/0NJJkA0rQkzIL26zWlLp6f
____________________________________________

Young Roddy McCorley (with Tommy Makem) - Live at Carnegie Hall, New York, NY - November 1962  by  The Clancy Brothers
https://open.spotify.com/track/1fbAJc87QxIRQ85Gowfhi3
____________________________________________

Don Giovanni, K. 527 Act 1 Scene 8: "Ho capito, signore sì!"  by  Wolfgang Amadeus Mozart
https://open.spotify.com/track/0st1RfQvfLWMXIrd7Gx1u0
____________________________________________

Bim Bom - Live At Carnegie Hall/1964  by  Stan Getz
https://open.spotify.com/track

# Functions that made the code running

In [23]:
#(1) Get audiofeatures of your playlist

playlist_id="2cqiYaM8wr1fFYFZbCXEwh"
username="11131870089"


def get_audio_features(username, playlist_id):
    features_dict = {}
    results = sp.user_playlist_tracks(username,playlist_id)
    tracks = results['items']
    while results['next']:
        results = sp.next(results)
        tracks.extend(results['items'])
    for track in tracks:
        afeatures = sp.audio_features(track["track"]["uri"])
        features_dict[track['track']['name']] = afeatures
    return features_dict

track_feature=get_audio_features(username, playlist_id)

In [24]:
#(2) Get audiofeatures_df of your playlist

def get_audio_features_df(features_dict):
    
    songs=list(track_feature.keys())
    #song

    song_df=pd.json_normalize(track_feature['Cola'][0])
    song_df.set_index('id',inplace=True)
    song_df=song_df[['danceability', 'energy', 'loudness', 'speechiness','instrumentalness', 'liveness', 'valence', 'tempo']]

    for song in songs:
        df=pd.json_normalize(features_dict[song])
        df.set_index('id',inplace=True)
        df=df[['danceability', 'energy', 'loudness', 'speechiness', 'instrumentalness', 'liveness', 'valence', 'tempo']]

        song_df=pd.concat([song_df,df])

    return song_df

song_df=get_audio_features_df(track_feature)

In [25]:
# TEST (2) Get audiofeatures_df of your playlist

song_df=get_audio_features_df(track_feature)
#song_df

In [26]:
#(3) get cluster for your playlist

def cluster_playlist(song_df):
    #scaling
    scaler_loaded = pickle.load(open('scaler.pickle','rb'))
    song_df_scaled=scaler_loaded.transform(song_df)

    #modeling
    model_loaded = joblib.load('model1.pkl')
    cluster_predict=model_loaded.predict(song_df_scaled)
    cluster_predict = pd.DataFrame(cluster_predict,index=song_df.index)
    song_df_cluster=song_df
    song_df_cluster["cluster"]=cluster_predict
    
    return song_df_cluster

In [27]:
# TEST (3) get cluster for your playlist

song_df_cluster=cluster_playlist(song_df)
#song_df_cluster

In [28]:
df_original=song_df_cluster

df_original.to_csv("/Users/carolinvogt/Becoming_Data_Analyst/Week_07/Day_04/spotify_project/song_df_cluster.csv")
print("sucessfully safed")



sucessfully safed


In [38]:
# (4) Get song recommendations

def recommendations(song_df_cluster): 
    
    uniques=list(song_df_cluster["cluster"].unique())

    db=pd.read_csv("/Users/carolinvogt/Becoming_Data_Analyst/Week_07/Day_04/spotify_project/kaggle_clustered_160k.csv")
    db.set_index('id',inplace=True)

    print()
    print("Here are my recommendations for you improving your playlist: ")
    for unique in range(len(uniques)):
        cluster_unique=db.loc[db["cluster"]==uniques[unique]].sample()
        song_id=cluster_unique.index[0]
        print()
        print(f"For cluster {uniques[unique]} I recommend:")
        print()
        print(sp.track(song_id)["name"]," by ",sp.track(song_id)["artists"][0]["name"])
        print(sp.track(song_id)["external_urls"]["spotify"])
        print("____________________________________________")

In [39]:
# TEST (4) Get song recommendations

recommendations(song_df_cluster)


Here are my recommendations for you improving your playlist: 

For cluster 10 I recommend:

Setareh (2017)  by  Navab Jalil
https://open.spotify.com/track/1maBBcntFMya5pTORWUzGG
____________________________________________

For cluster 15 I recommend:

Mamabo - Remastered  by  Edmundo Ros
https://open.spotify.com/track/0cXjW3YEPBoml9e3GlCj03
____________________________________________

For cluster 7 I recommend:

Aaron's Party (Come Get It)  by  Aaron Carter
https://open.spotify.com/track/4fF34Yf5KsfUbCORyLGpEj
____________________________________________

For cluster 25 I recommend:

My Fair Lady: I'm an Ordinary Man  by  Frederick Loewe
https://open.spotify.com/track/575jS2dwFSvYcX3aKOnMFl
____________________________________________

For cluster 22 I recommend:

(I've Got A) Golden Ticket - From "Willy Wonka & The Chocolate Factory" Soundtrack  by  Jack Albertson
https://open.spotify.com/track/0wHZkxYMe8gnelXhjKWymG
____________________________________________

For cluster 17 I re